In [1]:
import gym
import numpy as np
import time
from IPython import display
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [3]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [4]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            break
    
    return v_values

In [5]:
def policy_improvement(env, V, policy, max_iters=500, gamma=0.9):
    # loop through each state in the environment
    for state in range(env.observation_space.n):
        action_values = np.zeros(env.action_space.n)

        # loop through each action
        for action in range(env.action_space.n):
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                  action_values[action] += prob * (reward + gamma * V[next_state])

            # select the best action
            best_action = np.argmax(action_values)
            policy[state] = best_action

    return policy

In [6]:
def policy_iteration(env, game, max_iters=500, gamma=0.9):
    start = time.time()
    # Start with a random policy
    if game == "frozen-lake":
        rd = random.randint(0,3)
    else:
        rd = random.randint(0,5)

    policy = np.zeros(env.observation_space.n) + rd

    # Repeat until convergence or critical number of iterations reached
    for i in range(int(max_iters)):

        prev_policy = np.copy(policy)

        # Policy eveluation
        V = policy_evaluation(env, policy, max_iters)

        # Policy improvement
        policy = policy_improvement(env, V, policy, max_iters)

        # check convergence
        if np.all(np.isclose(policy, prev_policy)):
            print(f'Converged at {i}-th iteration.')
            print("optimal_policy:", policy)
            break

    end = time.time()
    average_time = (end - start) / i
    print("Total execution time:", end - start)
    print("Average excution time:", average_time)

    return policy, end - start, average_time, i

In [7]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    print("Optimal_policy:", policy)
    
    return policy

In [8]:
def value_iteration(env, max_iters=500, gamma=0.9):
    start = time.time()
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    optimal_policy = policy_extraction(env, v_values, gamma=0.9)    

    end = time.time()
    average_time = (end - start) / i
    print("Total execution time:", end - start)
    print("Average excution time:", average_time)
    
    return optimal_policy, end - start, average_time, i

In [9]:
from IPython.display import clear_output

# Hàm đếm số iteration lớn và nhỏ nhất khi hội tụ, thời gian trung bình chạy mỗi thuật toán.
def multiple_run_function(env, num_run, fn="value_iteration"):
    total_time = []
    i_converged = []
    for i in range(num_run):
        if fn == "policy_iteration":
            _, total_ex_time, average_ex_time, i_th = policy_iteration(env, "frozen-lake", max_iters=500, gamma=0.9)

        else:
            _, total_ex_time, average_ex_time, i_th = value_iteration(env, max_iters=500, gamma=0.9)

        i_converged.append(i_th)
        total_time.append(total_ex_time)

    i_converged_max, i_converged_min = max(i_converged), min(i_converged)
    avg_time_run = sum(total_time) / len(total_time)

    clear_output(wait=True)

    print(f'{fn}: max converged i-th: {i_converged_max}, min converged i-th: {i_converged_min}')
    print(f"Average time {num_run} run: {avg_time_run}")

In [10]:
env_FrozenLakeV0 = gym.make('FrozenLake-v0')

In [11]:
print("observation_space:", env_FrozenLakeV0.observation_space.n)
print("env.action_space:", env_FrozenLakeV0.action_space.n)

observation_space: 16
env.action_space: 4


In [12]:
optimal_policy_1, total_time_1, average_time_1, _ = policy_iteration(env_FrozenLakeV0, "frozen-lake", max_iters=500, gamma=0.9)

Converged at 4-th iteration.
optimal_policy: [0. 3. 0. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]
Total execution time: 0.04585003852844238
Average excution time: 0.011462509632110596


In [13]:
play_multiple_times(env_FrozenLakeV0, optimal_policy_1, 1000)

Number of successes: 743/1000
Average number of steps: 36.89636608344549


In [14]:
num_run = 500
fn = 'policy_iteration'
multiple_run_function(env_FrozenLakeV0, num_run, fn)

policy_iteration: max converged i-th: 5, min converged i-th: 1
Average time 500 run: 0.038978415966033936


In [15]:
optimal_policy_2, total_time_2, average_time_2, _ = value_iteration(env_FrozenLakeV0, max_iters=500, gamma=0.9)

Converged at 79-th iteration.
Optimal_policy: [0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
Total execution time: 0.052996158599853516
Average excution time: 0.0006708374506310572


In [16]:
play_multiple_times(env_FrozenLakeV0, optimal_policy_2, 1000)

Number of successes: 738/1000
Average number of steps: 37.31436314363144


In [17]:
num_run = 500
fn = 'value_iteration'
multiple_run_function(env_FrozenLakeV0, num_run, fn)

value_iteration: max converged i-th: 79, min converged i-th: 79
Average time 500 run: 0.050828081130981445


In [18]:
# Đối với toy game FrozenLake-v0
# 1. Policy iteration
#     policy iteration cho kết quả hội tụ rất nhanh từ 1 đến 5 iteration. 
#     Thời gian trung bình cho mỗi lần thực thi policy iteration cho kết quả là 0.03748822069168091 giây
# 2. Value iteration
#     Value iteration cho kết quả hội tụ lâu hơn policy iteration, phải mất 79 iteration để value iteration hội tụ ra kết quả (so với 1-5 iteration)
#     Thời gian trung bình cho mỗi lần thực thi value iteration cho kết quả là 0.054465441226959226 giây, giá trị này lớn hơn thời gian của policy iteration.
#     Tuy nhiên, nếu so sánh thời gian thực thi ở mỗi iteration thì value iteration nhanh hơn policy iteration (0.00064 < 0.0215 s)
#
# Số lần thành công khi thực thi nhiều lần hàm play_multiple_times() của policy iteration lớn hơn, trung bình bước nhỏ hơn của value iteration.
# -> với toy game FrozenLake-v0 thuật toán Policy iteration cho kết quả tốt hơn


In [19]:
env_FrozenLake8x8V0 = gym.make('FrozenLake8x8-v0')

In [20]:
print("observation_space:", env_FrozenLake8x8V0.observation_space.n)
print("env.action_space:", env_FrozenLake8x8V0.action_space.n)

observation_space: 64
env.action_space: 4


In [49]:
optimal_policy_1, total_time_1, average_time_1, _ = policy_iteration(env_FrozenLake8x8V0, "frozen-lake", max_iters=500, gamma=0.9)

Converged at 9-th iteration.
optimal_policy: [3. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 1. 3. 3. 0. 0. 2. 3. 2. 1.
 3. 3. 3. 1. 0. 0. 2. 1. 3. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 1. 1. 0.]
Total execution time: 0.21987295150756836
Average excution time: 0.024430327945285372


In [56]:
play_multiple_times(env_FrozenLake8x8V0, optimal_policy_1, 1000)

Number of successes: 730/1000
Average number of steps: 70.23013698630137


In [23]:
num_run = 500
fn = 'policy_iteration'
multiple_run_function(env_FrozenLake8x8V0, num_run, fn)

policy_iteration: max converged i-th: 9, min converged i-th: 2
Average time 500 run: 0.16743802547454834


In [51]:
optimal_policy_2, total_time_2, average_time_2, _ = value_iteration(env_FrozenLake8x8V0, max_iters=500, gamma=0.9)

Converged at 117-th iteration.
Optimal_policy: [3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0]
Total execution time: 0.17261433601379395
Average excution time: 0.001475336205246102


In [52]:
play_multiple_times(env_FrozenLake8x8V0, optimal_policy_2, 1000)

Number of successes: 751/1000
Average number of steps: 73.5392809587217


In [26]:
num_run = 500
fn = 'value_iteration'
multiple_run_function(env_FrozenLake8x8V0, num_run, fn)

value_iteration: max converged i-th: 117, min converged i-th: 117
Average time 500 run: 0.21420513343811035


In [27]:
# Đối với toy game FrozenLake8x8-v0
# 1. Policy iteration
#     policy iteration cho kết quả hội tụ rất nhanh từ 2 đến 9 iteration. 
#     Thời gian trung bình cho mỗi lần thực thi policy iteration cho kết quả là 0.16743802547454834 giây
# 2. Value iteration
#     Value iteration cho kết quả hội tụ lâu hơn policy iteration, phải mất 117 iteration để value iteration hội tụ ra kết quả (so với 2-9 iteration, gấp hơn 13 lần)
#     Thời gian trung bình cho mỗi lần thực thi value iteration cho kết quả là 0.21420513343811035 giây, giá trị này lớn hơn thời gian của policy iteration.
#     Tuy nhiên, nếu so sánh thời gian thực thi ở mỗi iteration thì value iteration nhanh hơn policy iteration (0.0022 < 0.044 s)
#
# Số lần thành công và trung bình bước khi thực thi nhiều lần hàm play_multiple_times() của policy iteration và value iteration chênh lệch nhau tùy lần chạy, khó xác định thuật toán nào hơn.
# -> với toy game FrozenLake8x8-v0, nếu dựa vào thời gian thực thi thì thuật toán policy iteration cho kết quả tốt hơn.


In [28]:
env_TaxiV3 = gym.make('Taxi-v3')

In [29]:
print("observation_space:", env_TaxiV3.observation_space.n)
print("env.action_space:", env_TaxiV3.action_space.n)

observation_space: 500
env.action_space: 6


In [30]:
optimal_policy_1, total_time_1, average_time_1, _ = policy_iteration(env_TaxiV3, "frozen-lake", max_iters=500, gamma=0.9)

Converged at 16-th iteration.
optimal_policy: [4. 4. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 3. 3. 3. 3.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 4. 4. 4. 4. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 5. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 0. 0. 0. 0. 2. 2. 2. 2.
 1. 2. 0. 2. 1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 2. 1. 2. 3. 2.
 3. 3. 3. 3. 1. 1. 1. 1. 3. 3. 3. 3. 2. 2. 2. 2. 3. 1. 3. 2. 3. 3. 3. 3.
 1. 1. 1. 1. 3. 3. 3. 3. 0. 0. 0. 0. 3. 1. 3. 0. 3. 3. 3. 3. 1. 1. 1. 1.
 3. 3. 3. 3. 0. 0. 0. 0. 3. 1. 3. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 1

In [31]:
play_multiple_times(env_TaxiV3, optimal_policy_1, 1000)

Number of successes: 1000/1000
Average number of steps: 13.203


In [32]:
num_run = 500
fn = 'policy_iteration'
multiple_run_function(env_TaxiV3, num_run, fn)

policy_iteration: max converged i-th: 16, min converged i-th: 16
Average time 500 run: 2.6518786878585816


In [33]:
optimal_policy_2, total_time_2, average_time_2, _= value_iteration(env_TaxiV3, max_iters=500, gamma=0.9)

Converged at 116-th iteration.
Optimal_policy: [4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 1 2 0 2 1 1
 1 1 2 2 2 2 3 3 3 3 2 2 2 2 1 2 3 2 3 3 3 3 1 1 1 1 3 3 3 3 2 2 2 2 3 1 3
 2 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0 3 1 3 0 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0
 3 1 3 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 4 4 4 4 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [34]:
play_multiple_times(env_TaxiV3, optimal_policy_2, 1000)

Number of successes: 1000/1000
Average number of steps: 12.967


In [35]:
num_run = 500
fn = 'value_iteration'
multiple_run_function(env_TaxiV3, num_run, fn)

value_iteration: max converged i-th: 116, min converged i-th: 116
Average time 500 run: 1.3672375087738038


In [36]:
# Đối với toy game Taxi-v3
# 1. Policy iteration
#     policy iteration cho kết quả hội tụ rất nhanh 16 iteration (trong cả 500 lần chạy). 
#     Thời gian trung bình cho mỗi lần thực thi policy iteration cho kết quả là 2.6518786878585816 giây
# 2. Value iteration
#     Value iteration cho kết quả hội tụ lâu hơn policy iteration, phải mất 116 iteration để value iteration hội tụ ra kết quả (so với 16 iteration)
#     Thời gian trung bình cho mỗi lần thực thi value iteration cho kết quả là 1.3672375087738038 giây, giá trị này nhỏ hơn thời gian của policy iteration.
#
# Số lần thành công khi thực thi hàm play_multiple_times() của policy iteration và value iteration bằng nhau, số trung bình bước của value iteration nhỏ hơn.
# -> với toy game Taxi-v0, tuy số iteration để hội tụ nhiều hơn nhưng thời gian thực thi ngắn hơn, số bước trung bình nhỏ hơn, và số lần thành công như nhau
#     thì thuật toán value iteration tốt hơn trong trường hợp game này


In [ ]:
# Nhìn chung đối với game ít trạng thái thì thuật toán policy iteration cho hiệu quả hơn (thời gian hội tụ nhanh) so với value iteration. Tuy nhiên khi số lượng
# trạng thái nhiều hơn thì value iteration lại hiệu quả hơn, số iteration để hội tụ có thể nhiều hơn policy iteration nhưng thời gian thực thi ngắn hơn và
# kết quả khi chơi thành công không chênh lệch so với policy iteration.